## 청경채 성장 예측 AI 경진대회(데이콘)
https://dacon.io/competitions/official/235961/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic') 

from datetime import timedelta

import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

import os

In [2]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
import cv2

In [3]:
path = 'train_input/'
train_inputs = os.listdir('train_input/')

for file_name in train_inputs:
    globals()[file_name.split('.')[0]+'_input'] = pd.read_csv(path+file_name)
    

In [4]:
path = 'train_target/'
train_targets = os.listdir('train_target/')

for file_name in train_targets:
    globals()[file_name.split('.')[0]+'_target'] = pd.read_csv(path+file_name)
    

In [5]:
import random
import pandas as pd
import numpy as np
import os
import glob

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

import warnings
warnings.filterwarnings(action='ignore') 

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Scale

In [7]:
# feature_names = ['내부온도관측치','내부습도관측치','CO2관측치', 
#                  '펌프상태', '일간누적분무량', '내부유동팬상태', '외부환기팬상태',
#                  '난방부하', '냉방부하', '총추정광량','백색광추정광량', '적색광추정광량', '청색광추정광량']
feature_names = ['외부환기팬작동남은시간', '레드 LED동작강도', '냉방부하', '내부습도관측치', '외부환기팬상태', '화이트 LED동작강도', '백색광추정광량', '총추정광량']

original_cols = ['시간', '내부온도관측치', '내부습도관측치', 'CO2관측치', 'EC관측치', '외부온도관측치', '외부습도관측치',
                 '펌프상태', '펌프작동남은시간', '최근분무량', '일간누적분무량', '냉방상태', '냉방작동남은시간',
                 '난방상태', '난방작동남은시간', '내부유동팬상태', '내부유동팬작동남은시간', '외부환기팬상태',
                 '외부환기팬작동남은시간', '화이트 LED상태', '화이트 LED작동남은시간', '화이트 LED동작강도',
                 '레드 LED상태', '레드 LED작동남은시간', '레드 LED동작강도', '블루 LED상태', '블루 LED작동남은시간',
                 '블루 LED동작강도', '카메라상태', '냉방온도', '난방온도', '기준온도', '난방부하', '냉방부하',
                 '총추정광량', '백색광추정광량', '적색광추정광량', '청색광추정광량']

In [8]:
# df_final = CASE_01_input

# for i in range(2,59):
#     df=eval(f'CASE_{str(i).zfill(2)}_input')
#     df.columns= original_cols
#     df_final = pd.concat([df_final,df], ignore_index=True)
    
# scaler = MinMaxScaler()
# scaler.fit(df_final.drop('시간', axis=1))

In [9]:
# 각 칼럼별 중앙값으로 채우기
def filling_na(df):
    #칼럼 맞추고, 스케일링 파트
#     df.columns= original_cols
#     df = df.drop('시간', axis=1)
    
    #결측치 채우기
    df_col_list = df.columns.tolist()
    df_describe = df.describe().fillna(0)
    
    for col in df_col_list:
        if col == '시간':
            continue
        else:
            df[col] = df[col].fillna(df_describe[col]['50%'])
    
    return df

In [10]:
def remove_outlier(df, cols):
    for col in cols:
        if col == '시간':
            continue
        else:
            outlier_idx = df[col].idxmax()
            df.loc[outlier_idx, col] = np.nan
            df[col] = df[col].fillna(method='pad')
            
            if df[col].isnull().sum() != 0:
                df[col] = df[col].fillna(0)
    return df

In [11]:
# 스케일링함 수
def scaling_MinMax(df):
    df = scaler.transform(df)
    df = pd.DataFrame(df, columns=original_cols[1:])
    
    return df

In [12]:
feature = ['내부온도관측치', '내부습도관측치', 'CO2관측치', '외부온도관측치', '외부습도관측치', '최근분무량',
            '일간누적분무량', '화이트 LED동작강도', '레드 LED동작강도', '블루 LED동작강도', '난방부하', '냉방부하',
            '총추정광량', '백색광추정광량', '적색광추정광량', '청색광추정광량','target']

In [13]:
eval(f'CASE_{str(1).zfill(2)}_input').columns
# original_cols

Index(['시간', '내부온도관측치', '내부습도관측치', 'CO2관측치', 'EC관측치', '외부온도관측치', '외부습도관측치',
       '펌프상태', '펌프작동남은시간', '최근분무량', '일간누적분무량', '냉방상태', '냉방작동남은시간', '난방상태',
       '난방작동남은시간', '내부유동팬상태', '내부유동팬작동남은시간', '외부환기팬상태', '외부환기팬작동남은시간',
       '화이트 LED상태', '화이트 LED작동남은시간', '화이트 LED동작강도', '레드 LED상태', '레드 LED작동남은시간',
       '레드 LED동작강도', '블루 LED상태', '블루 LED작동남은시간', '블루 LED동작강도', '카메라상태', '냉방온도',
       '난방온도', '기준온도', '난방부하', '냉방부하', '총추정광량', '백색광추정광량', '적색광추정광량',
       '청색광추정광량'],
      dtype='object')

In [14]:
def groupbyday(i, test=True):
    temp = eval(f'CASE_{str(i).zfill(2)}_input')
    temp.columns = original_cols
    
    temp = filling_na(temp)
    
    temp.dropna(subset=['시간'],inplace=True)
    temp['시간'] = pd.to_datetime(temp['시간'])
    temp['년월일'] = temp['시간'].dt.year.astype(str) + '-' + temp['시간'].dt.month.astype(str) + '-' + temp['시간'].dt.day.astype(str)
    
    temp['년월일'] = pd.to_datetime(temp['년월일'])
    temp_mean = temp.groupby('년월일').mean()
    temp_mean = temp_mean.reset_index(drop=False)
    time_col = temp_mean['년월일']
    temp_mean.drop('년월일', axis=1, inplace=True)
    temp_mean['target'] = temp_mean.index
    if test:
        temp_mean['시간'] = time_col
        return temp_mean

    df = pd.DataFrame(columns=feature)
    scaler = MinMaxScaler()
    scaler.fit(temp_mean)
    df_sam = scaler.transform(temp_mean)
    df_sam = pd.DataFrame(df_sam, columns=temp_mean.columns)

    df_sam['시간']=time_col

    df = pd.concat([df, df_sam])
    return df

In [15]:
train = [groupbyday(i, test=False) for i in range(1,48)]
test = [groupbyday(i) for i in range(48,59)]

In [16]:
test[0]

,내부온도관측치,내부습도관측치,CO2관측치,EC관측치,외부온도관측치,외부습도관측치,펌프상태,펌프작동남은시간,최근분무량,일간누적분무량,...,난방온도,기준온도,난방부하,냉방부하,총추정광량,백색광추정광량,적색광추정광량,청색광추정광량,target,시간
0,21.107500,51.917986,546.794792,1.731846,21.532838,29.540607,2.372917,0.891667,500.522514,3565.948569,...,17.416646,18.854144,2.685215,18.337882,179.485971,146.011438,12.371928,21.102604,0,2022-04-08
1,21.514236,53.103715,488.920833,1.738840,22.595278,30.872153,2.372917,1.262500,694.195007,5953.846903,...,17.416641,18.854140,2.680708,20.463855,179.561839,146.039371,12.383420,21.139047,1,2022-04-09
2,21.795278,54.907271,485.293056,1.748795,22.441389,33.319306,2.512500,1.202083,620.785840,5322.392306,...,17.416681,18.854180,2.450713,21.903053,179.472377,145.968465,12.382271,21.121642,2,2022-04-10
3,22.457917,62.478375,466.813194,1.750013,23.422986,42.940275,2.234028,0.831250,489.207500,3628.361590,...,17.416653,18.854151,1.715155,25.216747,179.504416,146.004992,12.400083,21.099341,3,2022-04-11
4,22.788055,68.345382,474.870139,1.750594,23.858055,51.352639,2.372917,1.214583,661.607410,5568.956396,...,17.416657,18.854156,1.500200,26.866203,179.516988,146.010364,12.371928,21.134696,4,2022-04-12
5,22.199146,69.650004,483.341667,1.788605,21.828264,59.463622,2.372917,1.184028,653.770847,5565.515090,...,17.416737,18.854241,2.545000,23.962319,179.534064,146.023256,12.389166,21.121642,5,2022-04-13
6,21.438055,66.363713,500.545139,1.813837,20.133736,53.392165,2.372917,1.141667,630.279021,4994.580861,...,17.416705,18.854211,3.171308,20.132625,179.507151,146.000695,12.389166,21.117290,6,2022-04-14
7,21.068923,62.483906,498.867361,1.829901,20.240299,41.216624,2.512500,1.112500,588.028562,4947.471528,...,17.402704,18.839385,2.947219,18.332485,179.093969,145.696657,12.367906,21.024279,7,2022-04-15
8,21.036062,59.429941,481.411111,1.852926,20.647257,33.639590,2.234028,0.932639,503.853611,4126.618986,...,17.416797,18.854306,3.106590,18.144485,179.471410,146.026479,12.347221,21.097709,8,2022-04-16
9,21.022396,59.236384,479.647917,1.846207,15.023958,23.145368,2.372917,0.906944,513.896021,3841.252160,...,17.174875,18.597702,2.600196,19.018860,168.102804,136.035115,11.900770,19.995176,9,2022-04-17


In [17]:
df2 = pd.DataFrame(columns=['시간','rate'])

temp = [pd.concat([df2, eval(f'CASE_{str(i).zfill(2)}_target')]) for i in range(1,59)]    
    
for df2 in temp:
    df2['시간'] = pd.to_datetime(df2['시간']) 
    df2['시간'] = df2['시간'].dt.strftime('%Y-%m-%d')
    df2['시간'] = df2['시간'].astype('str')

    
for df, df2 in zip((train + test), temp):
    df['시간'] = df['시간'].astype('str')
    df = df.merge(df2, on='시간', how='left').dropna()

In [18]:
for i in range(47):
    train[i] = train[i].merge(temp[i], on='시간', how='left').dropna()

for i in range(11):
    test[i] = test[i].merge(temp[47+i], on='시간', how='left').dropna()

In [19]:
for df in train:
    try:
        train_df = pd.concat([train_df,df])
    except:
        train_df = df
        
train_df.reset_index(drop=True, inplace=True)

In [20]:
for df in test:
    try:
        test_df = pd.concat([test_df,df])
    except:
        test_df = df

test_df.reset_index(drop=True, inplace=True)

In [21]:
import xgboost as xgb

In [46]:
feature = ['내부온도관측치', '내부습도관측치', 'CO2관측치', '외부온도관측치', '외부습도관측치', '최근분무량',
            '일간누적분무량', '화이트 LED동작강도', '레드 LED동작강도', '블루 LED동작강도', '난방부하', '냉방부하',
            '총추정광량', '백색광추정광량', '적색광추정광량', '청색광추정광량']
# feature = original_cols
target = 'rate'

In [47]:
train_df

,내부온도관측치,내부습도관측치,CO2관측치,외부온도관측치,외부습도관측치,최근분무량,일간누적분무량,화이트 LED동작강도,레드 LED동작강도,블루 LED동작강도,...,레드 LED상태,레드 LED작동남은시간,블루 LED상태,블루 LED작동남은시간,카메라상태,냉방온도,난방온도,기준온도,시간,rate
0,0.016595,0.126161,0.135204,0.000000,0.055119,0.926623,0.930234,1.000000,0.987779,0.380569,...,0.200249,0.956841,0.105318,0.951031,1.0,0.997122,0.996803,0.009467,2021-02-18,0.50000
1,0.000000,0.128636,0.028126,0.069107,0.232157,0.902882,0.800004,0.994020,0.986323,0.380400,...,0.000000,0.954986,0.000000,0.960637,1.0,0.997314,0.997017,0.009479,2021-02-19,0.66667
2,0.468532,0.000000,0.000000,0.357312,0.547471,0.997031,0.857877,0.995024,0.987066,0.380710,...,0.350249,0.954903,0.262772,0.963578,0.5,0.997436,0.997153,0.009486,2021-02-20,0.60000
3,0.748370,0.329593,0.340939,0.668620,0.643601,0.917805,0.811452,0.992481,0.988308,0.380569,...,0.450249,0.974923,0.368352,0.966088,0.5,0.996160,0.995735,0.009412,2021-02-21,-0.12500
4,0.761729,0.753206,0.377430,0.728365,0.861512,0.933065,0.894417,0.999442,0.988369,0.382118,...,0.300000,0.967640,0.315431,0.971915,0.5,0.977176,0.974649,0.008317,2021-02-22,1.42857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,0.947870,0.773458,0.447454,0.000000,0.000000,0.400671,0.774515,0.377713,0.000000,0.000000,...,0.985366,0.713403,0.979409,0.699526,0.0,0.356475,0.356475,0.356475,2022-04-21,0.22476
1442,0.907855,0.837682,0.426066,0.000000,0.000000,0.382887,0.818241,0.384782,0.000000,0.000000,...,0.964837,0.863507,0.973451,0.854953,0.0,0.377316,0.377316,0.377316,2022-04-22,0.21755
1443,0.919003,0.776975,0.470960,0.000000,0.000000,1.000000,0.827400,0.374470,0.000000,0.000000,...,0.970687,0.714059,0.973480,0.714590,1.0,0.356685,0.356685,0.356685,2022-04-23,0.23329
1444,0.944637,0.948501,0.466877,0.000000,0.000000,0.000000,0.532141,0.376299,0.000000,0.000000,...,0.970746,0.690336,0.973510,0.693032,0.0,0.356468,0.356468,0.356468,2022-04-24,0.11123


In [48]:
# feature = train_df.columns[:-2]
# target = train_df.columns[-1]

In [49]:
X_train = train_df[feature]
y_train = train_df[target]

X_test = test_df[feature]
y_test = test_df[target]

In [51]:
reg = xgb.XGBRegressor(n_estimators=400, early_stopping_rounds=50,
                      learning_rate=1e-3)
reg.fit(X_train,y_train,
        eval_set = [(X_train,y_train),(X_test, y_test)],
       verbose=True)

[14:27:43] WARNING: /tmp/xgboost/src/learner.cc:547: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:1.65169	validation_1-rmse:1.17972
[1]	validation_0-rmse:1.65087	validation_1-rmse:1.17968
[2]	validation_0-rmse:1.65004	validation_1-rmse:1.17963
[3]	validation_0-rmse:1.64922	validation_1-rmse:1.17959
[4]	validation_0-rmse:1.64839	validation_1-rmse:1.17955
[5]	validation_0-rmse:1.64757	validation_1-rmse:1.17951
[6]	validation_0-rmse:1.64675	validation_1-rmse:1.17947
[7]	validation_0-rmse:1.64593	validation_1-rmse:1.17942
[8]	validation_0-rmse:1.64511	validation_1-rmse:1.17938
[9]	validation_0-rmse:1.64428	validation_1-rmse:1.17934
[10]	validation_0-rmse:1.64346	validation_1-rmse:1.17930
[11]	validation_0-rmse:1.64264	v

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, early_stopping_rounds=50,
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.001, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=400, n_jobs=4,
             num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [52]:
X_train

,내부온도관측치,내부습도관측치,CO2관측치,외부온도관측치,외부습도관측치,최근분무량,일간누적분무량,화이트 LED동작강도,레드 LED동작강도,블루 LED동작강도,난방부하,냉방부하,총추정광량,백색광추정광량,적색광추정광량,청색광추정광량
0,0.016595,0.126161,0.135204,0.000000,0.055119,0.926623,0.930234,1.000000,0.987779,0.380569,0.030215,0.000000,0.000000,0.000000,0.0,0.0
1,0.000000,0.128636,0.028126,0.069107,0.232157,0.902882,0.800004,0.994020,0.986323,0.380400,0.453228,0.047523,0.000000,0.000000,0.0,0.0
2,0.468532,0.000000,0.000000,0.357312,0.547471,0.997031,0.857877,0.995024,0.987066,0.380710,0.000000,0.326087,0.000000,0.000000,0.0,0.0
3,0.748370,0.329593,0.340939,0.668620,0.643601,0.917805,0.811452,0.992481,0.988308,0.380569,0.000000,0.653653,0.000000,0.000000,0.0,0.0
4,0.761729,0.753206,0.377430,0.728365,0.861512,0.933065,0.894417,0.999442,0.988369,0.382118,0.000000,0.670115,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,0.947870,0.773458,0.447454,0.000000,0.000000,0.400671,0.774515,0.377713,0.000000,0.000000,0.221385,0.661398,0.376861,0.376861,0.0,0.0
1442,0.907855,0.837682,0.426066,0.000000,0.000000,0.382887,0.818241,0.384782,0.000000,0.000000,0.547780,0.387063,0.384033,0.384033,0.0,0.0
1443,0.919003,0.776975,0.470960,0.000000,0.000000,1.000000,0.827400,0.374470,0.000000,0.000000,0.483408,0.479090,0.374470,0.374470,0.0,0.0
1444,0.944637,0.948501,0.466877,0.000000,0.000000,0.000000,0.532141,0.376299,0.000000,0.000000,0.214677,0.644499,0.375260,0.375260,0.0,0.0


In [53]:
reg.feature_importances_

array([8.5922167e-02, 4.6951461e-01, 1.7851943e-02, 3.3144222e-04,
       3.2350373e-01, 1.1319907e-03, 5.3623933e-03, 1.0397693e-03,
       7.1674382e-04, 3.0426981e-03, 7.1729661e-04, 6.1968365e-04,
       8.4222219e-04, 2.1919252e-03, 2.9172394e-02, 5.8038969e-02],
      dtype=float32)

In [54]:
pd.DataFrame(reg.feature_importances_, index = feature, columns=['importance']).sort_values(by='importance', ascending=False)

,importance
내부습도관측치,0.469515
외부습도관측치,0.323504
내부온도관측치,0.085922
청색광추정광량,0.058039
적색광추정광량,0.029172
CO2관측치,0.017852
일간누적분무량,0.005362
블루 LED동작강도,0.003043
백색광추정광량,0.002192
최근분무량,0.001132


In [55]:
def avg_day(temp):
    temp.dropna(subset=['시간'],inplace=True)
    temp['시간'] = pd.to_datetime(temp['시간'])
    temp['년월일'] = temp['시간'].dt.year.astype(str) + '-' + temp['시간'].dt.month.astype(str) + '-' + temp['시간'].dt.day.astype(str)
    
    temp['년월일'] = pd.to_datetime(temp['년월일'])
    temp = temp.groupby('년월일').mean()
    return temp

In [56]:
test_input = [pd.read_csv(f'./test_input/TEST_{str(num+1).zfill(2)}.csv') for num in range(6)]

In [57]:
test_target = [pd.read_csv(f'./test_target/TEST_{str(num+1).zfill(2)}.csv') for num in range(6)]

In [59]:
for i in range(6):
    df = avg_day(test_input[i])
    df['rate'] = reg.predict(df[feature])
    test_input[i] = df

In [60]:
for i in range(6):
    if test_target[i].shape[0] == test_input[i].shape[0]:
        test_target[i].rate = list(test_input[i].rate)
    else:
        dif = test_input[i].shape[0] - test_target[i].shape[0]
        test_target[i].rate = list(test_input[i][dif:].rate)

In [61]:
import glob
test_target_list = sorted(glob.glob('./test_target/*.csv'))

In [62]:
import zipfile
os.chdir("./test_target/")
submission = zipfile.ZipFile("../submission_0825.zip", 'w')
for path in test_target_list:
    path = path.split('/')[-1]
    submission.write(path)
submission.close()
os.chdir("..")